In [ ]:
import gc
import glob
import json
import random
from collections import defaultdict
from pathlib import Path
from tempfile import gettempdir

import numpy as np
import pandas as pd
import plotly
from plotly import graph_objects as go
import scipy as sp
import scipy.constants
from IPython.core.display import HTML, display
from numpy import ma

from common import *

In [ ]:
# Setup plotting

COLORS = plotly.colors.DEFAULT_PLOTLY_COLORS

In [ ]:
# Find the default output directory
OUTPUT_DIR = Path(gettempdir()) / "boltzmann_solver" / "simple"
if not OUTPUT_DIR.is_dir():
    OUTPUT_DIR = Path("/media/ssh/uni-josh/tmp/josh/boltzmann_solver/simple")
if not OUTPUT_DIR.is_dir():
    OUTPUT_DIR = Path("/media/ssh/bakerlab11/tmp/jellis/boltzmann_solver/simple")
print(f"Loading data from {OUTPUT_DIR}")

# N1F1

In [ ]:
data = read_csv(OUTPUT_DIR / "n1f1.csv")
# ul = pd.read_csv("/tmp/josh/ulysses/n1f1.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
fig = plot_asymmetry(data, ["H", "L1", "N1"])
fig.update_yaxes(range=[-30, 0])
fig.add_trace(go.Scatter(
    x=ul["beta"],
    y=ul["BL"].abs(),
    name="[U] B-L"
))
fig

In [ ]:
fig = plot_density(data, ["H", "L1", "N1"])
fig.add_trace(go.Scatter(
    x=ul["beta"],
    y=ul["N1"],
    name="[U] N1"
))
fig.add_trace(go.Scatter(
    x=ul["beta"],
    y=ul["N1eq"],
    name="[U] N1eq"
))
# fig.update_yaxes(type="log", range=[-20,1])
fig

# N1F3

In [ ]:
data = read_csv(OUTPUT_DIR / "n1f3.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "L2", "L3", "N1"])

In [ ]:
fig = plot_density(data, ["N1"])
fig.update_yaxes(range=[0, 2])

# N3F3

In [ ]:
data = read_csv(OUTPUT_DIR / "n3f3.csv")

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])
plot_integration(data)

In [ ]:
plot_asymmetry(data, ["H", "L1", "L2", "L3", "N1", "N2", "N3"])

In [ ]:
fig = plot_density(data, ["N1", "N2", "N3"])
fig.update_yaxes(range=[0, 2])

# Miscellaneous

## Evolution

In [ ]:
data, ptcls = read_evolution(OUTPUT_DIR / "evolution.json")

In [ ]:
go.Figure(
    data=[go.Scatter(name=p, x=data["beta"], y=data[p, "mass"]) for p in ptcls],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Mass [GeV]", type="log", exponentformat="power"),
    ),
)

In [ ]:
go.Figure(
    data=[
        go.Scatter(name=p, x=data["beta"], y=data[p, "mass"] * data["beta"])
        for p in ptcls
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(
            title="Mass / Temperatre", type="log", exponentformat="power"
        ),
    ),
)

In [ ]:
go.Figure(
    data=[
        go.Scatter(name=p, x=data["beta"], y=data[p, "width"] / data[p, "mass"])
        for p in ptcls
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Width / Mass", type="linear", exponentformat="power"),
    ),
)

## Gammas

In [ ]:
data = dict()
data["u"] = pd.read_csv("/tmp/josh/ulysses/n1f1.csv")
data["b"] = pd.read_csv(OUTPUT_DIR / "gamma.csv")
go.Figure(
    data=[
        go.Scatter(name="[U] decay", x=data["u"]["beta"], y=data["u"]["d1"]),
        go.Scatter(name="[U] washout", x=data["u"]["beta"], y=data["u"]["w1"]),
        go.Scatter(name="[B] decay", x=data["b"]["beta"], y=data["b"]["N1 ↔ L1 H"]),
        go.Scatter(name="[B] washout", x=data["b"]["beta"], y=data["b"]["L1 ↔ N1"]),
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Interaction Rate", type="log", range=[-30, 15], exponentformat="power"),
    ),
)

In [ ]:
data = dict()
data["u"] = pd.read_csv("/tmp/josh/ulysses/n1f1.csv")
data["b"] = pd.read_csv(OUTPUT_DIR / "asymmetry.csv")
go.Figure(
    data=[
        go.Scatter(name="[U] decay", x=data["u"]["beta"], y=data["u"]["epsd1"].abs()),
        go.Scatter(name="[B] decay", x=data["b"]["beta"], y=data["b"]["N1 ↔ L1 H"].abs()),
    ],
    layout=go.Layout(
        xaxis=go.layout.XAxis(
            title="Inverse Temperature [GeV⁻¹]", type="log", exponentformat="power",
        ),
        yaxis=go.layout.YAxis(title="Interaction Rate", type="log", range=[-40, 5], exponentformat="power"),
    ),
)